# Training models in Azure Databricks and deploying them on Azure ML

This notebook demostrates how to train models in Azure Databricks (or any Databricks implementation) and deploying those models on Azure ML. Two workflows are demostrated here depending on the level of integration you want to keep and how you want to do tracking:

1. **Scenario 1: Training on Azure Databricks while tracking experiments and models in Azure ML:** This example shows how to do training of models in Azure Databricks while doing all the tracking of experiments in Azure ML (instead of in the MLflow instance running on Azure Databricks). This will also allow you to seemessly deploy models to Azure ML deployment targets in the easiest way.
2. **Scenario 2: Training and tracking experiments in Azure Databricks with Model Registries in Azure ML:** This example shows how to do training and tracking of models in Azure Databricks. Tracking of experiments happens here in the MLflow instance running on Azure Databricks. However, model registries are kept on Azure ML to allow quick model's deployment from a centralized location and registry of models.

Read each scenario to know more about advantages and disadvantages of each approach.

## Before starting

To run this notebook ensure you have:
- A Databricks workspace with a compute with the following libraries:
  - xgboost
  - scikit-learn==1.1.1
  - pandas
  - numpy
  - mlflow
  - azureml-mlflow

Also, configure the following variables:

In [ ]:
aml_region = ""
subscription_id = ""
aml_resource_group = ""
aml_workspace_name = ""
adb_user_id = ""

In [ ]:
azureml_mlflow_uri = f"azureml://{aml_region}.api.azureml.ms/mlflow/v1.0/subscriptions/{subscription_id}/resourceGroups/{aml_resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{aml_workspace_name}"

In [ ]:
import warnings

warnings.simplefilter("ignore")

## Scenario 1: Training on Azure Databricks while tracking experiments and models in Azure ML

In [ ]:
import mlflow

mlflow.set_tracking_uri(azureml_mlflow_uri)

### Training a heart condition classifier

#### Configuring the experiment

Tracking of experiments will happen in Azure ML and hence we need to use the naming convention we generally use with MLflow. 

>Note that naming in Azure Databricks is different as you have to use the path to where the experiment will be saved. In Azure ML and in general MLflow this is not the case.

In [ ]:
mlflow.set_experiment(experiment_name="heart-condition-classifier")

> **About authentication:** Interactive Authentication or Device Authentication will be triggered when you can `set_experiment`. This is used to authenticate against Azure Machine Learning and be able to call the tracking API. If you are executing the code in the context of a job where interactive authentication is not possible, see the example `notebooks/using-mlflow/train-with-mlflow/xgboost_service_principal.ipynb` for an example about how to use a Service Principal to authenticate against Azure Machine Learning and MLflow.

Since all the tracking is happening in Azure ML, you can train and register models in the regular way you do with mlflow.

#### Exploring the data

In [ ]:
import pandas as pd

file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)
display(df)

As we can see, some of the variables are categorical. To make it simpler for our model to handle these values, let's use their encoded values:

In [ ]:
df["thal"] = df["thal"].astype("category").cat.codes

Let's split our dataset in train and test, so we can assess the performance of the model without overfitting the dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1), df["target"], test_size=0.3
)

#### Training a model

We are going to use autologging capabilities in MLflow to track parameters and metrics:

In [ ]:
mlflow.xgboost.autolog()

Let's create a simple classifier and train it:

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score

model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")

In [ ]:
with mlflow.start_run() as run:
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    print("Recall: %.2f%%" % (recall * 100.0))

### Registering the model in Azure ML

Since our experiments are being tracked in Azure ML, we can simply register models in the registry like this:

In [ ]:
mlflow.register_model(
    model_uri=f"runs:/{run.info.run_id}/model", name="databricks-heart-classifier"
)

## Scenario 2: Training and tracking experiments in Azure Databricks with Model Registries in Azure ML

In some cases you may want to keep doing tracking of experiments in the MLflow instance that comes with Azure Databricks. This is the case for instance of customers that were already using MLflow in Azure Databricks so they want to keep they existing experiments there. However, they may want to take adavantage of the deployment capabilities of Azure ML including managed inference solutions, no-code deployments, etc.

In this cases, it is possible to keep tracking of experiments on Azure Databricks while keeping you model's registered and deployed in Azure ML. This example shows you how to achieve this configuration:

### Configuring models' registry

MLflow allows you to segregate the instance where experiments are being tracked from the instance where models' are being tracked (or registered). The first one is referred to **Tracking URI** while the second one is referred as **Registry URI**. By default, both of them are set to the same value, and in Azure Databricks, both of them are set to "databricks" meaning that tracking and model registries will happen inside of the MLflow instance that Databricks runs for you.

We are going to track the experiments in Azure Databricks, but model registries will be held in Azure ML. This will allow us to manage the model's lifecycle - including deployments - in Azure ML.

In [ ]:
import mlflow

mlflow.set_registry_uri(azureml_mlflow_uri)

#### Configuring the experiment

Tracking of experiments will happen in Azure Datbricks and hence we need to use the naming we use here.  

>Note that naming in Azure Databricks is different as you have to use the path to where the experiment will be saved.

In [ ]:
mlflow.set_experiment(
    experiment_name=f"/Users/{adb_user_id}/heart-condition-classifier"
)

Out[43]: <Experiment: artifact_location='', experiment_id='368f3886-e4e3-449a-bd1e-593c5b0e01ff', lifecycle_stage='active', name='heart-condition-classifier', tags={}>

#### Exploring the data

In [ ]:
import pandas as pd

file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)
display(df)

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0
56,1,2,120,236,0,0,178,0,0.8,1,0,normal,0
62,0,4,140,268,0,2,160,0,3.6,3,2,normal,1
57,0,4,120,354,0,0,163,1,0.6,1,0,normal,0
63,1,4,130,254,0,2,147,0,1.4,2,1,reversible,1
53,1,4,140,203,1,2,155,1,3.1,3,0,reversible,0


As we can see, some of the variables are categorical. To make it simpler for our model to handle these values, let's use their encoded values:

In [ ]:
df["thal"] = df["thal"].astype("category").cat.codes

Let's split our dataset in train and test, so we can assess the performance of the model without overfitting the dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1), df["target"], test_size=0.3
)

#### Training a model

We are going to use autologging capabilities in MLflow to track parameters and metrics:

In [ ]:
mlflow.xgboost.autolog()

Let's create a simple classifier and train it:

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score

model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")

In [ ]:
with mlflow.start_run() as run:
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    recall = reca
    ll_score(y_test, y_pred)

    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    print("Recall: %.2f%%" % (recall * 100.0))

### Registering the model in Azure ML

So far, our model is trained and tracked inside of the MLflow instance in Azure Databricks. Now we want to register this model in Azure ML to manage the life cicle there. However, if we try to register the model as we usually do using the sintax `mlflow.register_model(model_uri=f"runs:/{run.info.run_id}/model").` you will found an error. The reason why this is happening is related to where runs are being stored.

Right now runs are being stored in Azure Databricks and models in Azure ML. If you try to create a registered model from a Run, Azure ML don't have any way to guess how to get access to the runs, that are stored in a different service. because of that, you can't use `runs:/` URI for registering models.

To overcome this limitation, you have to register the model from the artifacts themselfs, which you can achieve by first downloading them.

In [ ]:
client = mlflow.tracking.MlflowClient()
model_path = client.download_artifacts(run.info.run_id, path="model")

`model_path` is a local path to the artifacts representing the MLmodel created. We can use this artifacts to register the model now:

> **Important:** Note that doing this has some implications. Since Azure ML knows nothing about the run that generated this model, lineage is lost from this point on. You can, although, store the RUN ID that generated this model in a tag in the registry for your reference.

In [ ]:
mlflow.register_model(
    model_uri=f"file://{model_path}", name="databricks-heart-classifier"
)

Registered model 'databricks-heart-classifier' already exists. Creating a new version of this model...
Warning: no run_link will be recorded with the model version because no run_id was given
2022/05/26 15:30:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: databricks-heart-classifier, version 1
Created version '1' of model 'databricks-heart-classifier'.
Out[28]: <ModelVersion: creation_timestamp=1653579024430, current_stage='None', description='', last_updated_timestamp=1653579024430, name='databricks-heart-classifier', run_id='', run_link='', source='azureml://artifacts/LocalUpload/220526T152844-ec7e8a19/model', status='READY', status_message='', tags={}, user_id='', version='1'>

Notice in the instruction above how the protocol is now `file://` instead of `runs:/`.

## Deploying models registered in Azure ML

Once a model is registered in Azure ML, you can deploy them using either the UI interface in Azure ML Studio, the Azure ML CLI v2 from a console, or the azureml-mlflow plugin for MLflow. Use the approach it best suites your needs. Here we will demostrate how to do that using the MLflow deployment plugin.

### Deploying models registered in Azure ML to Managed Inference

To make the deployment happen, you will need a deployment client. Deployments can be generated using both the Python API for MLflow or MLflow CLI. In both cases, a JSON configuration file needs to be indicated with the details of the deployment you want to achieve. The full specification of this configuration can be found at [Managed online deployment schema (v2)](https://docs.microsoft.com/en-us/azure/machine-learning/reference-yaml-deployment-managed-online).

In [ ]:
import json
from mlflow.deployments import get_deploy_client

# Create the deployment configuration.
deploy_config = {
    "instance_type": "Standard_DS2_v2",
    "instance_count": 1,
}

Write the deployment configuration into a file.

In [ ]:
deployment_config_path = "deployment_config.json"
with open(deployment_config_path, "w") as outfile:
    outfile.write(json.dumps(deploy_config))

#### Configuring the deployment client

In [ ]:
# Indicate to MLflow where we want to deploy
client = get_deploy_client(azureml_mlflow_uri)

In [ ]:
# Indicate to MLflow from where the models need to be pulled from. Currently, the source and target URLs need to be the same.
mlflow.set_tracking_uri(azureml_mlflow_uri)

#### Deploying the model

In [ ]:
# MLflow requires the deployment configuration to be passed as a dictionary.
config = {"deploy-config-file": deployment_config_path}
model_name = "databricks-heart-classifier"
model_version = 1

# define the model path and the name is the service name
# if model is not registered, it gets registered automatically and a name is autogenerated using the "name" parameter below
client.create_deployment(
    model_uri=f"models:/{model_name}/{model_version}",
    config=config,
    name="mymodel-mir-deployment",
)